In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


import os

os.environ["R_HOME"] = "/root/miniconda3/envs/R/lib/R"

import rpy2.robjects as objects
from rpy2.robjects.packages import importr

base = importr("base")
r_pROC = importr("pROC")
base._libPaths()[0]


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

/tmp/ipykernel_117967/1622204394.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [2]:
df_ifr = pd.read_csv(
    "../CathAI/data/DeepCORO/CathReport_MHI_Merged/2017-2021_CathReport_first_value_in_interval_IFR.csv"
)

In [3]:
## Define column where IFR was performed
import pandas as pd


def create_new_col(df):
    # List of column names to be checked
    col_names = [
        "D2",
        "D3",
        "RVG1",
        "RVG2",
        "S1",
        "Saphène ou mammaire",
        "bx",
        "diagonal",
        "dist_lad",
        "dist_lcx",
        "dist_rca",
        "lad",
        "lcx",
        "leftmain",
        "lvp",
        "marg_d",
        "mid_lad",
        "mid_rca",
        "om1",
        "om2",
        "om3",
        "pda",
        "posterolateral",
        "prox_rca",
    ]

    # Iterating over rows using DataFrame.apply() and lambda function
    df["ifr_performed"] = df.apply(
        lambda row: ", ".join([col for col in col_names if row[col] != -1.0]), axis=1
    )
    return df

In [4]:
df = create_new_col(df_ifr)
df["contains_leftmain"] = df["ifr_performed"].str.contains("leftmain")
df["contains_lad"] = df["ifr_performed"].str.contains("lad")
df["contains_lcx"] = df["ifr_performed"].str.contains("lcx")
df["contains_rca"] = df["ifr_performed"].str.contains("rca|pda|posterolateral")

In [5]:
display(df.contains_leftmain.value_counts())

False    474712
True       2583
Name: contains_leftmain, dtype: int64

### Merge with DICOMS Extracted

In [ ]:
df_dicoms = pd.read_csv(
    "../CathAI/data/DeepCORO/EXAMS_Extracted/2017-2021_dicom_extracted_mod.csv"
)

In [ ]:
# Identify common columns in df and df_dicoms, excluding 'path'
common_cols = [col for col in df.columns if col in df_dicoms.columns and col != "path"]

# Drop common columns from df_dicoms
df_dicoms_dropped = df_dicoms.drop(columns=common_cols)

# Perform left merge
df_merged = pd.merge(
    df, df_dicoms_dropped, left_on="DICOMPath", right_on="path", how="left"
)

In [ ]:
### Merge with views
df_angle_object = pd.read_csv(
    "../CathAI/data/DeepCORO/CATHAI_Extracted_Concatenated/DeepCORO_df_angle_object_dicom_2017-2021.csv"
)
### Drop Unnamed columns
df_angle_object = df_angle_object.loc[
    :, ~df_angle_object.columns.str.contains("^Unnamed")
]

In [ ]:
# Perform left merge
df_merged_angle_object = pd.merge(df_merged, df_angle_object, on="path", how="left")

In [ ]:
display(df_merged_angle_object.object_value.value_counts())

In [ ]:
display(df_merged_angle_object.contains_leftmain.value_counts())
display(df_merged_angle_object.contains_lad.value_counts())
display(df_merged_angle_object.contains_lcx.value_counts())
display(df_merged_angle_object.contains_rca.value_counts())

In [ ]:
# df_f.to_csv("data/df_f.csv", index=False)
# df_merged_angle_object.to_csv("data/df_merged_angle_object_IFR.csv", index=False)

Sure, here's a markdown table based on the labels' definitions:

| Label | Name         | 'primary' range | 'secondary' range |
|-------|--------------|-----------------|-------------------|
| 0     | RAO Cranial  | -60 to -15      | 15 to 50          |
| 1     | AP Cranial   | -15 to 15       | 15 to 50          |
| 2     | LAO Cranial  | 15 to 60        | 15 to 50          |
| 3     | RAO Straight | -60 to -15      | -15 to 15         |
| 4     | AP           | -15 to 15       | -15 to 15         |
| 5     | LAO Straight | 15 to 60        | -15 to 15         |
| 6     | RAO Caudal   | -60 to -15      | -50 to -15        |
| 7     | AP Caudal    | -15 to 15       | -50 to -15        |
| 8     | LAO Caudal   | 15 to 60        | -50 to -15        |
| 9     | LAO Lateral  | -110 to -70     | -15 to 15         |
| 10    | RAO Lateral  | 70 to 110       | -15 to 15         |
| 11    | Other        | Not applicable  | Not applicable    |

Note: The ranges in the 'primary' and 'secondary' columns are inclusive.

## Generate videos

In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))

from downloadAvi import extract_avi_metadata as avi_meta

In [ ]:
df_merged_angle_object = pd.read_csv("data/df_merged_angle_object_IFR.csv")

In [ ]:
df_filtered = df_merged_angle_object[
    (df_merged_angle_object["contains_leftmain"] == True)(
        df_merged_angle_object["contains_lad"] == True
    )
    | (df_merged_angle_object["contains_lcx"] == True)
    | (df_merged_angle_object["contains_rca"] == True)
]
display(df_filtered.describe())
# df_filtered.to_csv("data/df_merged_angle_object_IFR_performed_2017-2021.csv")

In [78]:
avi_meta.extract_avi_and_metadata(
    "data/df_merged_angle_object_IFR_performed_2017-2021_left_main.csv",
    data_type="ANGIO",
    destinationFolder="deepIFR/",
    dataFolder="data/",
)

Processing rows: 100%|██████████| 2583/2583 [30:09<00:00,  1.43it/s]  
/volume/DicomVideoProcessing/downloadAvi/extract_avi_metadata.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["FPS"] = metadata[fps_col_names].bfill(axis=1).iloc[:, 0]


,brand,sex,FPS,NumberOfFrames,date,study_time,series_time,birthdate,color_format,mrn,StudyID,StudyInstanceUID,SeriesInstanceUID,dicom_path,FileName,primary_angle,secondary_angle,width,height,pixel_spacing,distance_source_to_detector,distance_source_to_patient,estimated_radiographic_magnification_factor,table_motion,radiation_setting,image_pixel_spacing,Split
0,,,15.0,88.0,20171205,075514.000000,082250,19550223,MONOCHROME2,120242280555174550372158401634647117727,'1.3.46.670589.28.265405998058.201712051322578224412211511',2.16.124.113611.1.118.1.1.4806147,'1.3.46.670589.28.265405998058.20171205132250050629.2.2',/media/data1/ravram/DeepCORO/2017/120242280555174550372158401634647117727/2.16.124.113611.1.118.1.1.4806147/1.3.46.670589.28.265405998058.20171205132250050629.2.2.dcm,deepIFR/2.16.124.113611.1.118.1.1.4806147_1.3.46.670589.28.265405998058.20171205132250050629.2.2.dcm.avi,43.4,-23.200,512,512,NaN,1116.0,765.000000,NaN,NaN,GR,"[0.258390625, 0.258390625]",inference
1,,,15.0,88.0,20171205,075514.000000,082325,19550223,MONOCHROME2,120242280555174550372158401634647117727,'1.3.46.670589.28.265405998058.201712051323334734512211511',2.16.124.113611.1.118.1.1.4806147,'1.3.46.670589.28.265405998058.20171205132325622631.2.2',/media/data1/ravram/DeepCORO/2017/120242280555174550372158401634647117727/2.16.124.113611.1.118.1.1.4806147/1.3.46.670589.28.265405998058.20171205132325622631.2.2.dcm,deepIFR/2.16.124.113611.1.118.1.1.4806147_1.3.46.670589.28.265405998058.20171205132325622631.2.2.dcm.avi,-14.6,-23.200,512,512,NaN,1091.0,765.000000,NaN,NaN,GR,"[0.258390625, 0.258390625]",inference
2,,,15.0,84.0,20171205,075514.000000,082349,19550223,MONOCHROME2,120242280555174550372158401634647117727,'1.3.46.670589.28.265405998058.201712051323576684612211511',2.16.124.113611.1.118.1.1.4806147,'1.3.46.670589.28.265405998058.20171205132349656633.2.2',/media/data1/ravram/DeepCORO/2017/120242280555174550372158401634647117727/2.16.124.113611.1.118.1.1.4806147/1.3.46.670589.28.265405998058.20171205132349656633.2.2.dcm,deepIFR/2.16.124.113611.1.118.1.1.4806147_1.3.46.670589.28.265405998058.20171205132349656633.2.2.dcm.avi,-10.1,39.600,512,512,NaN,1134.0,765.000000,NaN,NaN,GR,"[0.258390625, 0.258390625]",inference
3,,,15.0,89.0,20171205,075514.000000,082421,19550223,MONOCHROME2,120242280555174550372158401634647117727,'1.3.46.670589.28.265405998058.201712051324298544712211511',2.16.124.113611.1.118.1.1.4806147,'1.3.46.670589.28.265405998058.20171205132421171635.2.2',/media/data1/ravram/DeepCORO/2017/120242280555174550372158401634647117727/2.16.124.113611.1.118.1.1.4806147/1.3.46.670589.28.265405998058.20171205132421171635.2.2.dcm,deepIFR/2.16.124.113611.1.118.1.1.4806147_1.3.46.670589.28.265405998058.20171205132421171635.2.2.dcm.avi,42.5,23.900,512,512,NaN,1050.0,765.000000,NaN,NaN,GR,"[0.258390625, 0.258390625]",inference
4,,,15.0,79.0,20171205,075514.000000,082455,19550223,MONOCHROME2,120242280555174550372158401634647117727,'1.3.46.670589.28.265405998058.201712051325027814812211511',2.16.124.113611.1.118.1.1.4806147,'1.3.46.670589.28.265405998058.20171205132455180638.2.2',/media/data1/ravram/DeepCORO/2017/120242280555174550372158401634647117727/2.16.124.113611.1.118.1.1.4806147/1.3.46.670589.28.265405998058.20171205132455180638.2.2.dcm,deepIFR/2.16.124.113611.1.118.1.1.4806147_1.3.46.670589.28.265405998058.20171205132455180638.2.2.dcm.avi,23.1,-39.801,512,512,NaN,1190.0,765.000000,NaN,NaN,GR,"[0.258390625, 0.258390625]",inference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,,,10.0,3.0,20211215,145223.000000,151910.000000,19561228,MONOCHROME2,42641673287655615184909697886539709288,'1.3.12.2.1107.5.4.5.135214.30000021121411131593700001868.512',2.16.124.113611.1.118.1.1.5980975,'1.3.12.2.1107.5.4.5.135214.30000021121411131593700001869',/media/data1/ravram/DeepCORO/2021/42641673287655615184909697886539709288/2.16.124.113611.1.118.1.1.5980975/1.3.12.2.1107.5.4.5.135214.3000002112141113159

## Create dataframe of 3 videos per artery with 3 different views

In [6]:
import pandas as pd
from tqdm import tqdm


def create_labels(df, primary, secondary, label_column, label2_column):
    label_list = []
    label2_list = []
    for _, row in tqdm(df.iterrows()):
        p = row[primary]
        s = row[secondary]
        if (-60 <= p <= -15) and (15 <= s <= 50):
            label_list.append(0)
            label2_list.append("RAO Cranial")
        elif (-15 <= p <= 15) and (15 <= s <= 50):
            label_list.append(1)
            label2_list.append("AP Cranial")
        elif (15 <= p <= 60) and (15 <= s <= 50):
            label_list.append(2)
            label2_list.append("LAO Cranial")
        elif (-60 <= p <= -15) and (-15 <= s <= 15):
            label_list.append(3)
            label2_list.append("RAO Straight")
        elif (-15 <= p <= 15) and (-15 <= s <= 15):
            label_list.append(4)
            label2_list.append("AP")
        elif (15 <= p <= 60) and (-15 <= s <= 15):
            label_list.append(5)
            label2_list.append("LAO Straight")
        elif (-60 <= p <= -15) and (-50 <= s <= -15):
            label_list.append(6)
            label2_list.append("RAO Caudal")
        elif (-15 <= p <= 15) and (-50 <= s <= -15):
            label_list.append(7)
            label2_list.append("AP Caudal")
        elif (15 <= p <= 60) and (-50 <= s <= -15):
            label_list.append(8)
            label2_list.append("LAO Caudal")
        elif (-110 <= p <= -70) and (-15 <= s <= 15):
            label_list.append(9)
            label2_list.append("LAO Lateral")
        elif (70 <= p <= 110) and (-15 <= s <= 15):
            label_list.append(10)
            label2_list.append("RAO Lateral")
        else:
            label_list.append(11)
            label2_list.append("Other")

    df[label_column] = label_list
    df[label2_column] = label2_list
    return df


def convert_seconds_to_time(df, time_column, new_time_column):
    """
    Converts the specified column from seconds to datetime format.

    Parameters:
    df : DataFrame
        The DataFrame which contains the column to be converted.
    time_column : str
        The name of the column to be converted.

    Returns:
    DataFrame
        The DataFrame with the converted column.
    """
    df[new_time_column] = pd.to_timedelta(df[time_column], unit="s")
    return df


def format_time(df, time_column):
    """
    Formats the specified timedelta column to 'hh:mm:ss' format.

    Parameters:
    df : DataFrame
        The DataFrame which contains the column to be formatted.
    time_column : str
        The name of the column to be formatted.

    Returns:
    DataFrame
        The DataFrame with the formatted column.
    """
    df[time_column] = (
        df[time_column]
        .dt.components[["hours", "minutes", "seconds"]]
        .astype(str)
        .agg(":".join, axis=1)
    )
    return df

In [7]:
df_ifr_videos = pd.read_csv("data/df_merged_angle_object_IFR_performed_2017-2021.csv")

In [8]:
df_ifr_videos.loc[df_ifr_videos["series_time"] < 0, "series_time"] = None
df_ifr_videos = convert_seconds_to_time(
    df_ifr_videos, "series_time", "series_time_human"
)
df_ifr_videos = format_time(df_ifr_videos, "series_time_human")

df_ifr_videos.loc[df_ifr_videos["study_time"] < 0, "study_time"] = None
df_ifr_videos = convert_seconds_to_time(df_ifr_videos, "study_time", "study_time_human")
df_ifr_videos = format_time(df_ifr_videos, "study_time_human")

/root/miniconda3/lib/python3.8/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/root/miniconda3/lib/python3.8/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [9]:
# df_ifr_metadata_1 = pd.read_csv("data/df_merged_angle_object_IFR_performed_2017-2021_left_main.csv_metadata_extracted.csv")
# df_ifr_metadata_2 = pd.read_csv("data/df_merged_angle_object_IFR_performed_2017-2021.csv_metadata_extracted.csv")
# df_ifr_metadata = pd.concat([df_ifr_metadata_1, df_ifr_metadata_2])
# Remove Unnamed: 0 columns
# df_ifr_metadata = df_ifr_metadata.loc[:, ~df_ifr_metadata.columns.str.contains('^Unnamed')]
# df_ifr_metadata.to_csv('data/df_merged_angle_object_IFR_performed_2017-2021_full_metadata_extracted.csv')

In [10]:
df_ifr_metadata = pd.read_csv(
    "data/df_merged_angle_object_IFR_performed_2017-2021_full_metadata_extracted.csv"
)

/tmp/ipykernel_113873/2273603541.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ifr_metadata = pd.read_csv(


In [11]:
## merged df_ifr_videos and df_ifr_metadata on dicom_path and path but remove from df_ifr_metadata the column StudyInstanceUID
df_ifr_metadata = df_ifr_metadata.drop(
    columns=["StudyInstanceUID", "series_time", "study_time"]
)
df_ifr_videos_metadata = pd.merge(
    df_ifr_videos, df_ifr_metadata, right_on="dicom_path", left_on="path", how="inner"
)

## Remove columns starting with Unnamed df_ifr_videos_metadata
df_ifr_videos_metadata = df_ifr_videos_metadata.loc[
    :, ~df_ifr_videos_metadata.columns.str.contains("^Unnamed")
]

In [12]:
df_ifr_videos_metadata = create_labels(
    df_ifr_videos_metadata,
    "primary_angle",
    "secondary_angle",
    "angle_true_label",
    "angle_true_string_label",
)

34863it [00:02, 15344.31it/s]


In [19]:
def explode_and_label(df):
    # convert the 'ifr_performed' column to a list of segments
    df["ifr_performed"] = df["ifr_performed"].str.split(", ")

    # explode the DataFrame
    df = df.explode("ifr_performed")

    # create the 'label' column by looking up the IFR value in the appropriate column for each row
    df["label"] = df.apply(lambda row: row[row["ifr_performed"]], axis=1)

    return df


# use the function
df_ifr_videos_metadata = explode_and_label(df_ifr_videos_metadata)

In [20]:
## Create y_true_cat column df_ifr_videos_metadata if ifr is ≤0.89 == 1 or >0.89 == 0
df_ifr_videos_metadata["y_true_cat"] = np.where(
    df_ifr_videos_metadata["label"] <= 0.89, 1, 0
)
display(df_ifr_videos_metadata.y_true_cat.value_counts())

# df_ifr_videos_metadata.to_csv(
#   "data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles.csv"
# )

0    30990
1    18748
Name: y_true_cat, dtype: int64

In [21]:
display(df_ifr_videos_metadata.ifr_performed.value_counts())
display(df_ifr_videos_metadata.angle_value.value_counts())
display(df_ifr_videos_metadata.angle_true_label.value_counts())
display(df_ifr_videos_metadata.angle_true_string_label.value_counts())
display(df_ifr_videos_metadata.object_value.value_counts())

mid_lad                14700
lad                     9782
lcx                     6169
mid_rca                 4023
leftmain                3653
prox_rca                2801
dist_lcx                2344
dist_rca                1354
diagonal                1100
om2                      796
dist_lad                 781
om1                      597
bx                       510
D2                       458
pda                      305
posterolateral           196
om3                       71
RVG2                      36
Saphène ou mammaire       34
RVG1                      28
Name: ifr_performed, dtype: int64

 7.0     9106
 0.0     7789
 1.0     7625
 8.0     6371
 4.0     3826
 10.0    3304
 5.0     2495
 9.0     2474
-1.0     1864
 6.0     1489
 3.0      954
 2.0      669
 11.0     113
Name: angle_value, dtype: int64

1     15399
5      9471
6      5542
7      4843
8      3753
2      3331
3      2459
0      2034
11     1646
4       606
10      591
9        63
Name: angle_true_label, dtype: int64

AP Cranial      15399
LAO Straight     9471
RAO Caudal       5542
AP Caudal        4843
LAO Caudal       3753
LAO Cranial      3331
RAO Straight     2459
RAO Cranial      2034
Other            1646
AP                606
RAO Lateral       591
LAO Lateral        63
Name: angle_true_string_label, dtype: int64

 5.0     13422
 6.0     10721
 10.0     9668
 1.0      6243
 9.0      5252
-1.0      1864
 2.0       557
 7.0       160
 8.0       111
 3.0        52
 4.0        25
 0.0         4
Name: object_value, dtype: int64

In [22]:
## Enumerate all of df_ifr_videos_metadata.angle_true_string_label and add them to list
angle_true_string_label_list = (
    df_ifr_videos_metadata.angle_true_string_label.unique().tolist()
)
display(angle_true_string_label_list)

['LAO Straight',
 'RAO Caudal',
 'RAO Cranial',
 'LAO Cranial',
 'LAO Caudal',
 'RAO Straight',
 'AP Cranial',
 'AP Caudal',
 'AP',
 'RAO Lateral',
 'LAO Lateral',
 'Other']

In [8]:
def get_sample_videos(df):
    arteries_conditions = {
        "contains_rca": {
            "y_pred_cat": 9.0,
            "ifr_performed": "rca|pda|posterolateral",
            "angle_true_string_label": [
                "LAO Straight",
                "RAO Caudal",
                "RAO Cranial",
                "LAO Cranial",
                "LAO Caudal",
                "RAO Straight",
                "AP Cranial",
                "AP Caudal",
                "AP",
                "RAO Lateral",
                "LAO Lateral",
                "Other",
            ],
        },
        "contains_lad": {
            "y_pred_cat": 5.0,
            "ifr_performed": "lad",
            "angle_true_string_label": [
                "AP Cranial",
                "AP " "LAO Caudal",
                "RAO Cranial",
                "AP Cranial",
                "LAO Lateral",
                "RAO Lateral",
            ],
        },
        "contains_lcx": {
            "y_pred_cat": 5.0,
            "ifr_performed": "lcx",
            "angle_true_string_label": ["LAO Cranial", "LAO Caudal", "AP Caudal", "AP"],
        },
        "contains_leftmain": {
            "y_pred_cat": 5.0,
            "ifr_performed": "leftmain",
            "angle_true_string_label": [
                "LAO Straight",
                "AP",
                "RAO Straight",
                "RAO Caudal",
                "AP Caudal",
                "LAO Caudal",
            ],
        },
    }

    result_df = pd.DataFrame()

    # Assuming 'examID' is the column in your df that represents different exams
    for ExamID, group_df in df.groupby("ExamID"):
        print(f"Processing ExamID: {ExamID}")
        for artery, condition in arteries_conditions.items():
            condition_met_df = group_df[
                (group_df["y_pred_cat"] == condition["y_pred_cat"])
                & (group_df["ifr_performed"].str.contains(condition["ifr_performed"]))
                & (
                    group_df["angle_true_string_label"].isin(
                        condition["angle_true_string_label"]
                    )
                )
            ]
            if not condition_met_df.empty:
                sample_df = condition_met_df.nsmallest(3, "series_time")
                result_df = pd.concat([result_df, sample_df])
            else:
                print(
                    f"No data found that satisfies the conditions for {artery} in examID: {ExamID}"
                )

    return result_df

In [6]:
## I regenerated the object_vALue using the video based moidel
result_df = pd.read_csv(
    "../Oread/data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_predictions_wide_fixed_merged_with_previous.csv"
)

/tmp/ipykernel_117967/2796091845.py:2: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df = pd.read_csv('../Oread/data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_predictions_wide_fixed_merged_with_previous.csv')


In [9]:
new_df = get_sample_videos(result_df)

Processing ExamID: 20215
No data found that satisfies the conditions for contains_rca in examID: 20215
No data found that satisfies the conditions for contains_lcx in examID: 20215
No data found that satisfies the conditions for contains_leftmain in examID: 20215
Processing ExamID: 20314
No data found that satisfies the conditions for contains_rca in examID: 20314
No data found that satisfies the conditions for contains_lcx in examID: 20314
No data found that satisfies the conditions for contains_leftmain in examID: 20314
Processing ExamID: 20540
No data found that satisfies the conditions for contains_rca in examID: 20540
No data found that satisfies the conditions for contains_lad in examID: 20540
No data found that satisfies the conditions for contains_leftmain in examID: 20540
Processing ExamID: 20663
No data found that satisfies the conditions for contains_rca in examID: 20663
No data found that satisfies the conditions for contains_lad in examID: 20663
No data found that satisfie

In [11]:
display(new_df.describe())

,Unnamed: 0.2,0,1,2,3,4,5,6,7,8,9,10,y_pred_cat,Unnamed: 0.1,Unnamed: 0,D2,D3,RVG1,RVG2,S1,Saphène ou mammaire,bx,diagonal,dist_lad,dist_lcx,dist_rca,lad,lcx,leftmain,lvp,marg_d,mid_lad,mid_rca,om1,om2,om3,pda,posterolateral,prox_rca,Patient_ID_Resume,Patient_ID_c-find,ExamID,StudyDate,StudyTime,PatientID,year,study_time,series_time,fps,frame_time,angle_count,angle_value,length,object_count,object_value,FPS,NumberOfFrames,date,birthdate,primary_angle,secondary_angle,width,height,distance_source_to_detector,distance_source_to_patient,estimated_radiographic_magnification_factor,angle_true_label,label,y_true_cat
count,3598.000000,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3.598000e+03,3598.000000,3598.000000,3598.000000,3598.0,3598.0,3598.0,3598.0,3598.0,3598.0,3598.0,3598.0,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.0,3598.0,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3.598000e+03,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,1029.000000,1029.000000,3484.000000,3484.000000,3484.000000,3484.000000,3287.000000,3598.000000,3598.000000,3.598000e+03,3.598000e+03,3598.000000,3598.000000,3598.000000,3598.000000,3598.000000,3546.000000,2258.000000,3598.000000,3598.000000,3598.000000
mean,16754.032796,1.884643e-09,9.180985e-03,5.666066e-04,9.605399e-10,1.096424e-09,7.862854e-01,5.223380e-08,1.410414e-09,3.024989e-09,2.004207e-01,4.424812e-03,5.802668,25146.800723,17303.555031,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.957065,-0.876406,-0.924177,-0.529305,-0.710050,-0.867610,-1.0,-1.0,-0.241820,-0.761101,-0.968394,-0.966262,-0.996417,-0.983146,-0.987260,-0.808469,464909.274041,464909.274041,69679.672318,2.019899e+07,114920.723180,464909.274041,2019.827960,114920.723180,120087.971813,10.675413,101.232064,57.625718,3.177669,70.766935,47.146670,5.182841,11.336854,70.381601,2.019899e+07,1.952411e+07,1.587407,18.352684,512.426904,512.426904,1100.595859,754.189567,1.512904,2.699277,0.887854,0.354641
std,9721.384596,3.461606e-08,4.473101e-02,8.752523e-03,1.953751e-08,2.200090e-08,3.988995e-01,7.797620e-07,2.874150e-08,6.139676e-08,3.962856e-01,3.555518e-02,1.602220,14133.559460,10026.996425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277450,0.473770,0.372158,0.813463,0.694016,0.482646,0.0,0.0,0.917307,0.638084,0.245000,0.252381,0.081256,0.180853,0.155661,0.580642,142546.479677,142546.479677,20652.349387,1.053466e+04,28344.395255,142546.479677,1.056443,28344.395255,30342.033209,3.850260,27.673245,41.136042,3.474075,45.578833,34.715877,2.171659,3.788021,45.256975,1.053466e+04,1.000625e+05,25.027018,26.914756,14.780163,14.780163,276.685198,54.087213,0.125424,2.698686,0.154281,0.478471
min,3.000000,0.000000e+00,1.156327e-29,0.000000e+00,0.000000e+00,0.000000e+00,6.445160e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.841445e-39,5.000000,3.000000,3.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1207.000000,1207.000000,20215.000000,2.017102e+07,14906.000000,1207.000000,2017.000000,14906.000000,809.000000,7.000000,33.333333,11.000000,0.000000,19.000000,7.000000,1.000000,7.000000,19.000000,2.017102e+07,1.927071e+07,-99.000000,-46.000000,512.000000,512.000000,-1000.000000,567.416700,1.183742,0.000000,0.000000,0.000000
25%,8486.250000,3.530618e-25,5.117847e-11,2.299604e-19,3.001885e-26,9.763733e-26,9.636672e-01,1.983245e-22,1.091641e-25,1.088472e-25,4.049225e-20,4.584523e-15,5.000000,13585.750000,8799.250000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,385160.000000,385160.000000,54652.000000,2.019080e+07,92832.000000,385160.000000,2019.000000,92

In [325]:
# new_df.to_csv('data/df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_3_videos.csv')
# exploded_df.to_csv('data/exploded_df_merged_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_3_videos.csv')

In [24]:
import os
import tempfile

from IPython.display import HTML, Video, display
from ipywidgets import interactive
from moviepy.editor import VideoFileClip

# Create a temporary directory
temp_dir = tempfile.TemporaryDirectory()

# Select the first ExamID
exam_id = result_df["ExamID"].iloc[3300]

# Select the corresponding rows
rows = result_df[result_df["ExamID"] == exam_id]
video_data = rows[["FileName", "angle_true_string_label", "ifr_performed"]].values

# Convert .avi to .mp4 and save them in the temporary directory
for i in range(len(video_data)):
    clip = VideoFileClip(video_data[i][0])
    new_video_path = os.path.join(temp_dir.name, f"video_{i}.mp4")
    clip.write_videofile(new_video_path, codec="libx264")
    video_data[i][0] = new_video_path


def play_video(index):
    video_path, angle_label, ifr_performed = video_data[index]
    ifr_performed = ifr_performed.split(",")[
        0
    ]  # pick only the first value when multiple values are present
    ifr_value = rows.iloc[index][ifr_performed]
    display(
        HTML(f"<h2>Angle: {angle_label}, IFR: {ifr_performed}, Value: {ifr_value}</h2>")
    )
    display(Video(video_path, embed=True))


video_player = interactive(play_video, index=(0, len(video_data) - 1))

display(video_player)  # Manually display the widget

Moviepy - Building video /tmp/tmpt_0vk4bo/video_0.mp4.
Moviepy - Writing video /tmp/tmpt_0vk4bo/video_0.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpt_0vk4bo/video_0.mp4
Moviepy - Building video /tmp/tmpt_0vk4bo/video_1.mp4.
Moviepy - Writing video /tmp/tmpt_0vk4bo/video_1.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpt_0vk4bo/video_1.mp4
Moviepy - Building video /tmp/tmpt_0vk4bo/video_2.mp4.
Moviepy - Writing video /tmp/tmpt_0vk4bo/video_2.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpt_0vk4bo/video_2.mp4


interactive(children=(IntSlider(value=1, description='index', max=2), Output()), _dom_classes=('widget-interac…

In [128]:
# When you're done with the videos, you can clean up the temporary directory
temp_dir.cleanup()

### Generate dataframe for OREAD

In [15]:
def transform_rows(df, number_of_views=3):
    # Drop duplicates on StudyID and angle_true_string_label
    df = df.drop_duplicates(subset=["ExamID", "angle_true_string_label"])

    # Group by StudyID
    groups = df.groupby("ExamID")

    # Initialize an empty DataFrame to store the transformed data
    new_df = pd.DataFrame()

    # Loop through each group
    for name, group in groups:
        # Further group by 'ifr_performed'
        subgroups = group.groupby("ifr_performed")

        for subname, subgroup in subgroups:
            # Create a new record
            new_record = {}
            new_record["ExamID"] = name
            new_record["ViewCount"] = number_of_views
            new_record["Label"] = df.loc[
                (df["ExamID"] == name) & (df["ifr_performed"] == subname), "label"
            ].values[0]

            # Add required fields from df
            new_record["y_true_cat"] = subgroup.iloc[0]["y_true_cat"]
            new_record["PatientID"] = subgroup.iloc[0]["PatientID"]
            new_record["patient_id_anon"] = subgroup.iloc[0]["patient_id_anon"]
            new_record["ifr_performed"] = subgroup.iloc[0]["ifr_performed"]
            new_record["FPS"] = subgroup.iloc[0]["FPS"]

            if (
                len(subgroup) < number_of_views
            ):  # We need at least two records with different angles
                print(
                    f"ExamID {name} with ifr_performed {subname} has less unique angle_true_string_label entries."
                )
                # Select the first record from the subgroup three
                subgroup = pd.concat(
                    [subgroup.iloc[:1]] * number_of_views, ignore_index=True
                )
                new_record["unique_angles"] = False
            else:
                # Select the first two records from each subgroup
                subgroup = subgroup.iloc[:number_of_views]
                new_record["unique_angles"] = True

            # Set the filename and angle_true_string_label for the two records
            for i in range(number_of_views):
                new_record[f"FileName{i}"] = subgroup.iloc[i]["FileName"]
                new_record[f"angle_true_string_label{i}"] = subgroup.iloc[i][
                    "angle_true_string_label"
                ]

            # Append the new record to new_df
            new_df = new_df.append(new_record, ignore_index=True)

    return new_df

In [17]:
display(df_results_transformed.unique_angles.value_counts())

True     742
False    613
Name: unique_angles, dtype: int64

In [16]:
df_results_transformed = transform_rows(new_df, number_of_views=2)

ExamID 20215 with ifr_performed dist_lad has less than 3 unique angle_true_string_label entries.
ExamID 20314 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 20540 with ifr_performed dist_lcx has less than 3 unique angle_true_string_label entries.
ExamID 21248 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 21257 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 21285 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 21403 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 21778 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 22389 with ifr_performed dist_lcx has less than 3 unique angle_true_string_label entries.
ExamID 22436 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 22477 with ifr_performed mid_lad has less than

/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is de

ExamID 23124 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 24444 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 24474 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 24541 with ifr_performed dist_lad has less than 3 unique angle_true_string_label entries.
ExamID 24803 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 25056 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 25241 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 25332 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 25397 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 26176 with ifr_performed mid_rca has less than 3 unique angle_true_string_label entries.
ExamID 26345 with ifr_performed lad has less than 3 unique angl

/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is de

ExamID 26622 with ifr_performed dist_lcx has less than 3 unique angle_true_string_label entries.
ExamID 26691 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 26800 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 26817 with ifr_performed mid_rca has less than 3 unique angle_true_string_label entries.
ExamID 26890 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 26928 with ifr_performed dist_lad has less than 3 unique angle_true_string_label entries.
ExamID 28007 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 28100 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 31378 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 32766 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 33009 with ifr_performed mid_lad has less t

...

/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is de

ExamID 100782 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 100942 with ifr_performed mid_rca has less than 3 unique angle_true_string_label entries.
ExamID 100961 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 100981 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 101006 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 101067 with ifr_performed dist_lcx has less than 3 unique angle_true_string_label entries.
ExamID 101347 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 101361 with ifr_performed dist_lcx has less than 3 unique angle_true_string_label entries.
ExamID 101606 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 101683 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 101700 with ifr_performed mid_lad

/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is de

ExamID 101986 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 102072 with ifr_performed lad has less than 3 unique angle_true_string_label entries.
ExamID 102141 with ifr_performed lcx has less than 3 unique angle_true_string_label entries.
ExamID 102223 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 102396 with ifr_performed lcx has less than 3 unique angle_true_string_label entries.
ExamID 102489 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.
ExamID 102544 with ifr_performed mid_lad has less than 3 unique angle_true_string_label entries.


/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)
/tmp/ipykernel_117967/4138906382.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_record, ignore_index=True)


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Get list of unique patient IDs
patient_ids = df_results_transformed["PatientID"].unique()

# Split patient IDs into train, val and test
train_ids, temp_ids = train_test_split(patient_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.33, random_state=42)

# Create boolean masks to filter rows for each set
train_mask = df_results_transformed["PatientID"].isin(train_ids)
val_mask = df_results_transformed["PatientID"].isin(val_ids)
test_mask = df_results_transformed["PatientID"].isin(test_ids)

# Apply masks to split data into sets
df_train = df_results_transformed[train_mask]
df_train["Split"] = "train"
df_val = df_results_transformed[val_mask]
df_val["Split"] = "val"
df_test = df_results_transformed[test_mask]
df_test["Split"] = "test"

# Concatenate the three sets
df_split = pd.concat([df_train, df_val, df_test])

/tmp/ipykernel_117967/3875987253.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Split'] = 'train'
/tmp/ipykernel_117967/3875987253.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['Split'] = 'val'
/tmp/ipykernel_117967/3875987253.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [19]:
df_split.to_csv(
    "data/df_train_val_test_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_2_videos.csv"
)

In [20]:
df_split = pd.read_csv(
    "data/df_train_val_test_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_2_videos.csv"
)

In [31]:
display(df_split.head(n=5))

,Unnamed: 0,ExamID,ViewCount,Label,y_true_cat,PatientID,patient_id_anon,ifr_performed,FPS,unique_angles,FileName0,angle_true_string_label0,FileName1,angle_true_string_label1,Split
0,0,20215,2,0.79,1,359640,251939745006906217388592365402890035562,dist_lad,10.0,False,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,train
1,1,20314,2,0.87,1,514871,196794625596455989704322125142939703308,mid_lad,7.0,False,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.4430.20171019192243241.3.272.dcm.avi,AP Cranial,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.4430.20171019192243241.3.272.dcm.avi,AP Cranial,train
2,2,20540,2,0.88,1,531062,262757768794829980357284170132797989772,dist_lcx,15.0,False,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024210200273344.2.2.dcm.avi,AP Caudal,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024210200273344.2.2.dcm.avi,AP Caudal,train
3,4,21001,2,0.98,0,347727,243121862769937149156975823684925028261,prox_rca,15.0,True,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140401945071.2.2.dcm.avi,LAO Straight,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140429755073.2.2.dcm.avi,AP Cranial,train
4,6,21257,2,0.86,1,487659,312609248941937773694292021454368478471,lad,15.0,False,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106175826254177.2.2.dcm.avi,AP Cranial,../../../DicomVideoProcessing/deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106175826254177.2.2.dcm.avi,AP Cranial,train


In [32]:
import os
import shutil

import pandas as pd
from tqdm import tqdm

dst_folder = "../Oread/deepIFR/"

# Make sure the destination folder exists, if not, create it
if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

# Loop through all the rows in the DataFrame
for idx, row in tqdm(df_split.iterrows()):
    # Get filenames from the DataFrame row and remove the prefix
    filenames = [
        row["FileName0"].replace("../../../DicomVideoProcessing/", ""),
        row["FileName1"].replace("../../../DicomVideoProcessing/", ""),
    ]

    # Loop through all the files in the source directory
    for file in filenames:
        # Check if the source file exists
        if os.path.exists(file):
            # Construct full file path
            dst = os.path.join(dst_folder, os.path.basename(file))
            # Copy the file to destination dir
            shutil.copy(file, dst)
        else:
            print(f"File {file} not found.")

print("Files copied successfully!")

1355it [00:20, 67.11it/s] 

Files copied successfully!


In [38]:
display(df_split.head(n=5))

,Unnamed: 0,ExamID,ViewCount,Label,y_true_cat,PatientID,patient_id_anon,ifr_performed,FPS,unique_angles,FileName0,angle_true_string_label0,FileName1,angle_true_string_label1,FileName2,angle_true_string_label2,Split
0,0,20215,3,0.79,1,359640,251939745006906217388592365402890035562,dist_lad,10.0,False,deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,train
1,1,20314,3,0.87,1,514871,196794625596455989704322125142939703308,mid_lad,7.0,False,deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.2383.20171019192842326.3.288.dcm.avi,RAO Lateral,deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.2383.20171019192842326.3.288.dcm.avi,RAO Lateral,deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.2383.20171019192842326.3.288.dcm.avi,RAO Lateral,train
2,2,20540,3,0.88,1,531062,262757768794829980357284170132797989772,dist_lcx,15.0,False,deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024202943448311.2.2.dcm.avi,AP Caudal,deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024202943448311.2.2.dcm.avi,AP Caudal,deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024202943448311.2.2.dcm.avi,AP Caudal,train
3,4,21001,3,0.98,0,347727,243121862769937149156975823684925028261,prox_rca,15.0,False,deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140401945071.2.2.dcm.avi,LAO Straight,deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140401945071.2.2.dcm.avi,LAO Straight,deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140401945071.2.2.dcm.avi,LAO Straight,train
4,6,21257,3,0.86,1,487659,312609248941937773694292021454368478471,lad,15.0,False,deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106180057972186.2.2.dcm.avi,RAO Lateral,deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106180057972186.2.2.dcm.avi,RAO Lateral,deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106180057972186.2.2.dcm.avi,RAO Lateral,train


In [33]:
df_split["FileName0"] = "/volume/Oread/" + df_split["FileName0"].str.replace(
    "../../../DicomVideoProcessing/", ""
)
df_split["FileName1"] = "/volume/Oread/" + df_split["FileName1"].str.replace(
    "../../../DicomVideoProcessing/", ""
)

/tmp/ipykernel_117967/3778454599.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_split['FileName0'] = '/volume/Oread/' + df_split['FileName0'].str.replace('../../../DicomVideoProcessing/', '')
/tmp/ipykernel_117967/3778454599.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_split['FileName1'] = '/volume/Oread/' + df_split['FileName1'].str.replace('../../../DicomVideoProcessing/', '')


In [34]:
df_split.to_csv(
    "data/df_train_val_test_angle_object_IFR_performed_2017_with_metadata_and_angles_processed_filtered_2_videos.csv"
)

In [35]:
display(df_split.head(n=5))

,Unnamed: 0,ExamID,ViewCount,Label,y_true_cat,PatientID,patient_id_anon,ifr_performed,FPS,unique_angles,FileName0,angle_true_string_label0,FileName1,angle_true_string_label1,Split
0,0,20215,2,0.79,1,359640,251939745006906217388592365402890035562,dist_lad,10.0,False,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4770998_1.3.12.2.1107.5.4.5.135214.30000017101604421511400001839.dcm.avi,AP Cranial,train
1,1,20314,2,0.87,1,514871,196794625596455989704322125142939703308,mid_lad,7.0,False,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.4430.20171019192243241.3.272.dcm.avi,AP Cranial,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4772671_1.2.392.200036.9116.4430.20171019192243241.3.272.dcm.avi,AP Cranial,train
2,2,20540,2,0.88,1,531062,262757768794829980357284170132797989772,dist_lcx,15.0,False,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024210200273344.2.2.dcm.avi,AP Caudal,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4775588_1.3.46.670589.28.265405998058.20171024210200273344.2.2.dcm.avi,AP Caudal,train
3,4,21001,2,0.98,0,347727,243121862769937149156975823684925028261,prox_rca,15.0,True,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140401945071.2.2.dcm.avi,LAO Straight,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4781002_1.3.46.670589.28.3711508510107.20171101140429755073.2.2.dcm.avi,AP Cranial,train
4,6,21257,2,0.86,1,487659,312609248941937773694292021454368478471,lad,15.0,False,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106175826254177.2.2.dcm.avi,AP Cranial,/volume/Oread/deepIFR/2.16.124.113611.1.118.1.1.4785029_1.3.46.670589.28.3711508510107.20171106175826254177.2.2.dcm.avi,AP Cranial,train
